Import Library

In [1]:
import numpy as np
import pandas as pd

Importing & Parsing the dataset as ratings and movies details

In [9]:
ratingData = pd.io.parsers.read_csv('ratings.dat', 
             names=['user_id', 'movie_id', 'rating', 'time'],
             engine='python', delimiter='::')
movieData = pd.io.parsers.read_csv('movies.dat', 
             names=['movie_id', 'title', 'genre'],
             engine='python', delimiter='::')
print(ratingData)

UnicodeDecodeError: ignored

Create the ratings matrix of shape (m*u)

In [8]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1,] = ratingData.rating.values
print(ratingMatrix)

[[  5 247  32 ... 127   0   3]
 [ 48  58 214 ... 127   0   0]
 [ 48  74  19 ... 127   0   0]
 ...
 [204 114  46 ...   0   0   0]
 [156 255  46 ...   0   0   0]
 [123 253  46 ...   0   0   0]]


Subtract Mean off - Normalization

In [10]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[ -69.88642384  172.11357616  -42.88642384 ...   52.11357616
   -74.88642384  -71.88642384]
 [ -54.31258278  -44.31258278  111.68741722 ...   24.68741722
  -102.31258278 -102.31258278]
 [ -55.54387417  -29.54387417  -84.54387417 ...   23.45612583
  -103.54387417 -103.54387417]
 ...
 [ 161.15248344   71.15248344    3.15248344 ...  -42.84751656
   -42.84751656  -42.84751656]
 [ 117.75662252  216.75662252    7.75662252 ...  -38.24337748
   -38.24337748  -38.24337748]
 [  80.58410596  210.58410596    3.58410596 ...  -42.41589404
   -42.41589404  -42.41589404]]


Computing SVD

In [11]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Calculate cosine similarity, sort by most similar and return the top N

In [12]:
def similar(ratingData, movie_id, top_n):
  index = movie_id - 1 # Movie id starts from 1
  movie_row = ratingData[index, :]
  magnitude = np.sqrt(np.einsun('ij, ij -> i', ratingData, ratingData))
  similarity = np.dot (movie_row, ratingData.T) / (magnitude[index] + magnitude) 
  sort_indexes = np.argsort(-similarity) 
  return sort_indexes[:top_n]

Select K principal components to represent the movies, a movie_id to find recommendations and print the top_n results

In [14]:
k = 50
movie_id = 2 
top_n = 5

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie (0): \n'.format( 
    movieData[movieData.movie_id == movie_id].title.values[0])) 
for id in indexes + 1: 
  print(movieData[movieData.movie_id == id].title.values[0])

AttributeError: ignored